In [3]:
import mygene

In [6]:
def convert_gmt_entrez_to_symbols(input_file, output_file, query_chunk_size=1000):
    """
    Convert an input GMT-like file of Entrez Gene IDs to HGNC gene symbols.

    Each line in the input is expected to have at least two metadata columns,
    followed by one or more Entrez Gene IDs (tab-delimited). The output file
    will mirror the input format but with gene symbols replacing Entrez IDs.

    Parameters
    ----------
    input_file : str
        Path to the input file containing Entrez Gene IDs.
    output_file : str
        Path to the output file where gene symbols will be written.
    query_chunk_size : int, optional
        Number of IDs to query per batch to MyGeneInfo (default=1000).
    """
    mg = mygene.MyGeneInfo()

    # Step 1: Read lines and collect unique IDs
    lines = []  # will store tuples of (metadata, gene_ids)
    unique_ids = set()

    with open(input_file, 'r') as infile:
        for lineno, line in enumerate(infile, start=1):
            parts = line.rstrip('\n').split('\t')
            if len(parts) < 3:
                # Skip lines without gene IDs
                continue

            metadata = parts[:2]
            gene_ids = parts[2:]
            lines.append((metadata, gene_ids))
            unique_ids.update(gene_ids)

    # Step 2: Query MyGeneInfo in chunks for performance
    id_to_symbol = {}
    all_ids = list(unique_ids)
    for i in range(0, len(all_ids), query_chunk_size):
        chunk = all_ids[i:i + query_chunk_size]
        results = mg.querymany(
            chunk,
            scopes='entrezgene',
            fields='symbol',
            species='human',
            as_dataframe=False
        )
        for res in results:
            q = str(res.get('query', ''))
            sym = res.get('symbol') or q
            id_to_symbol[q] = sym

    # Step 3: Write output by mapping IDs back to symbols
    with open(output_file, 'w') as outfile:
        for metadata, gene_ids in lines:
            symbols = [id_to_symbol.get(gid, gid) for gid in gene_ids]
            new_line = '\t'.join(metadata + symbols) + '\n'
            outfile.write(new_line)


In [8]:
t_input = "wikipathways-20250710-gmt-Homo_sapiens.gmt"
t_output = "wikipathways_2025_new.gmt"

convert_gmt_entrez_to_symbols(t_input, t_output, query_chunk_size=500)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequen

In [13]:
def compare_gmt_counts(file1, file2):
    """
    Compare two GMT-like files line by line, but only report the counts
    of gene IDs on each line (no value-level comparison).
    """
    # Read all lines from both files
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

    # Quick check: same number of lines?
    if len(lines1) != len(lines2):
        print(f"Line count mismatch: {len(lines1)} vs {len(lines2)}")
        return

    # Compare per‐line counts
    for i, (l1, l2) in enumerate(zip(lines1, lines2), start=1):
        # subtract 2 for the two metadata columns
        count1 = max(0, len(l1.strip().split('\t')) - 2)
        count2 = max(0, len(l2.strip().split('\t')) - 2)
        delta  = abs(count1 - count2)

        if count1 == count2:
            print(f"Line {i}: {count1} genes in both files (no difference)")
        else:
            print(
                f"Line {i}: file1 has {count1}, file2 has {count2} "
                f"(difference = {delta})"
            )

In [14]:
compare_gmt_counts("wikipathways-20250710-gmt-Homo_sapiens.gmt", "wikipathways_2025_new.gmt")

Line 1: 20 genes in both files (no difference)
Line 2: 12 genes in both files (no difference)
Line 3: 52 genes in both files (no difference)
Line 4: 105 genes in both files (no difference)
Line 5: 98 genes in both files (no difference)
Line 6: 16 genes in both files (no difference)
Line 7: 37 genes in both files (no difference)
Line 8: 30 genes in both files (no difference)
Line 9: 7 genes in both files (no difference)
Line 10: 8 genes in both files (no difference)
Line 11: 87 genes in both files (no difference)
Line 12: 67 genes in both files (no difference)
Line 13: 38 genes in both files (no difference)
Line 14: 19 genes in both files (no difference)
Line 15: 12 genes in both files (no difference)
Line 16: 34 genes in both files (no difference)
Line 17: 41 genes in both files (no difference)
Line 18: 13 genes in both files (no difference)
Line 19: 1 genes in both files (no difference)
Line 20: 11 genes in both files (no difference)
Line 21: 36 genes in both files (no difference)
Lin

In [15]:
import csv

infile  = "wikipathways_2025_new.gmt"
out1    = "wikipathways_2025_clean.gmt"
out2    = "wikipathways_2025_links.tsv"

with open(infile, newline='') as fin, \
     open(out1, "w", newline='') as f1, \
     open(out2, "w", newline='') as f2:

    reader = csv.reader(fin, delimiter="\t")
    writer1 = csv.writer(f1, delimiter="\t")
    writer2 = csv.writer(f2, delimiter="\t")

    # write header for TSV if you like:
    writer2.writerow(["name","source","ID","species","link","genes"])

    for row in reader:
        # split the first field on '%'
        name, source, gid, species = row[0].split("%")
        link = row[1]
        genes = row[2:]              # list of gene symbols

        # 1) output1.gmt: name [ID] + genes as separate columns
        writer1.writerow([f"{name} [{gid}]"] + genes)

        # 2) output2.tsv: name, source, ID, species, link, genes joined by commas
        writer2.writerow([name, source, gid, species, link, ",".join(genes)])
